# Load và tiền xử lý data

## Thư viện







In [ ]:
import os
import json
import pandas as pd
import sys
from google.colab import drive
from transformers import AdamW, TapexTokenizer, BartForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
import torch
import ast
from ast import literal_eval
from itertools import islice
from torch.nn.utils.rnn import pad_sequence
import pickle
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Mount Google Drive
drive.mount("/content/drive", force_remount=True)



Mounted at /content/drive


## **Lấy** data từ https://github.com/microsoft/HiTab

In [ ]:
# @title
import json
import pandas as pd
import os

base_dir = '/content/drive/MyDrive/Colab Notebooks/Hitab/data'
tables_dir = os.path.join(base_dir, 'tables', 'raw')
train_file = os.path.join(base_dir, 'train_samples.jsonl')
dev_file = os.path.join(base_dir, 'dev_samples.jsonl')
test_file = os.path.join(base_dir, 'test_samples.jsonl')

def load_data(file_path):
    samples = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                samples.append(json.loads(line.strip()))
    except FileNotFoundError:
        print(f"File {file_path} not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON in file {file_path}.")
    return pd.DataFrame(samples)

train_df = load_data(train_file)
dev_df = load_data(dev_file)
test_df = load_data(test_file)

df = pd.concat([train_df, dev_df, test_df], ignore_index=True)
df.head(20)


,id,table_id,table_source,sentence_id,sub_sentence_id,sub_sentence,question,answer,aggregation,linked_cells,answer_formulas,reference_cells_map
0,822d0514ad6c3058d70cffec53211837,0_totto74-1,totto,0,1,"in 2016 season, charles sims finished with 51 ...","in 2016 season, how many rushes did charles si...",[51.0],[none],"{'entity_link': {'top': {'rushes': {'(0, 4)': ...",[=E7],"{'E7': '(4, 4)'}"
1,249b7a7903f9dec2873b2ff4eafb8075,0_totto109-3,totto,0,1,schwartz was the manager of rot-weiß erfurt be...,"how many games did schwartz, the manager of ro...",[1.0],[none],"{'entity_link': {'top': {'matches': {'(1, 3)':...",[=E5],"{'E5': '(2, 4)'}"
2,57184793dfabcdd5a164e3c047a101e1,0_totto115-4,totto,0,1,"in total, craig finished his eleven nfl season...","in total, how many rushing yards did craig fin...",[8189.0],[none],"{'entity_link': {'top': {'yards': {'(1, 3)': '...",[=D16],"{'D16': '(13, 3)'}"
3,006e304c1c80127c12da2a041702ad17,1000,statcan,5757,1,a higher proportion of men than women were adm...,which had a higher proportion of admittances u...,[men],[pair-argmax],{'entity_link': {'top': {'family sponsorship p...,[=A17],"{'A17': '(14, 0)'}"
4,96db851bc4d88b027ffa428dea62e748,1001,statcan,5762,1,for immigrants from some african countries of ...,list african countries of origin that have mor...,"[cameroon, cote d'ivoire, senegal]",[greater_than],{'entity_link': {'top': {'admitted as a princi...,"[=A36, = A39, =A40]","{'A36': '(33, 0)', 'A39': '(36, 0)', 'A40': '(..."
5,46f5c6ff99aa9d0f5d04fd847593fc87,1002,statcan,5763,1,the black immigrant populations stand out for ...,which population has more women aged 25 to 59 ...,[black population],[pair-argmax],"{'entity_link': {'top': {}, 'left': {'the rest...",[=A5],"{'A5': '(2, 0)'}"
6,ea1e8a9b80bbc4bc75089edece4613ab,1002,statcan,5764,1,among black immigrant women aged 25 to 59 in 2...,how many percent of black immigrant women aged...,[29.2],[none],"{'entity_link': {'top': {'in 2016': {'(0, 4)':...",[=E9],"{'E9': '(6, 4)'}"
7,7a669cbe702769e1b9b5facc0244f24b,1002,statcan,5765,1,among black immigrant women aged 25 to 59 in 2...,among black immigrant women aged 25 to 59 in 2...,[9.8],[none],"{'entity_link': {'top': {'in 2016': {'(0, 4)':...",[=E13],"{'E13': '(10, 4)'}"
8,a39496f71bad9ca3352cf1cfee049e97,1002,statcan,5766,1,the highest proportion of lone mothers was fou...,which region of origion of the immigrants has ...,[caribbean and latin america],[argmax],"{'entity_link': {'top': {'in 2016': {'(0, 4)':...",[=A10],"{'A10': '(7, 0)'}"
9,41c09839ad631914ee8fb625dd81bc05,1003,statcan,5773,1,the proportions of black male immigrants from ...,which region of birth has fewer proportion of ...,[caribbean and latin america],[pair-argmin],{'entity_link': {'top': {'black male immigrant...,[=A6],"{'A6': '(3, 0)'}"


### Số lương table

In [ ]:
unique_train_tables = set(train_df['table_id'].unique()) if 'table_id' in train_df else set()
unique_dev_tables = set(dev_df['table_id'].unique()) if 'table_id' in dev_df else set()
unique_test_tables = set(test_df['table_id'].unique()) if 'table_id' in test_df else set()

all_unique_tables = unique_train_tables.union(unique_dev_tables).union(unique_test_tables)

print(f"Total tables all datasets: {len(all_unique_tables)}")

Total tables all datasets: 3597


In [ ]:
print(train_df.shape)
print(dev_df.shape)
print(test_df.shape)


(7417, 12)
(1671, 12)
(1584, 12)


### Thống kê các phép toán


In [ ]:
# @title
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'left')

train_df['aggregation'] = train_df['aggregation'].apply(tuple)
test_df['aggregation'] = test_df['aggregation'].apply(tuple)
dev_df['aggregation'] = dev_df['aggregation'].apply(tuple)

# Tính count và phần trăm cho train
train_counts = train_df['aggregation'].value_counts()
train_percentage = (train_counts / train_df['aggregation'].count()) * 100
train_summary = pd.DataFrame({'Train Count': train_counts, 'Train %': train_percentage})

# Tính count và phần trăm cho test
test_counts = test_df['aggregation'].value_counts()
test_percentage = (test_counts / test_df['aggregation'].count()) * 100
test_summary = pd.DataFrame({'Test Count': test_counts, 'Test %': test_percentage})

# Tính count và phần trăm cho dev
dev_counts = dev_df['aggregation'].value_counts()
dev_percentage = (dev_counts / dev_df['aggregation'].count()) * 100
dev_summary = pd.DataFrame({'Dev Count': dev_counts, 'Dev %': dev_percentage})

comparison_table = pd.concat([train_summary, test_summary, dev_summary], axis=1)

comparison_table = comparison_table.round(2)

comparison_table = comparison_table.reset_index(drop=False)
comparison_table


,aggregation,Train Count,Train %,Test Count,Test %,Dev Count,Dev %
0,"(none,)",5208.0,70.22,1133.0,71.53,1195.0,71.51
1,"(pair-argmax,)",494.0,6.66,93.0,5.87,107.0,6.40
2,"(div,)",331.0,4.46,76.0,4.80,91.0,5.45
3,"(opposite,)",249.0,3.36,49.0,3.09,44.0,2.63
4,"(argmax,)",226.0,3.05,54.0,3.41,64.0,3.83
5,"(sum,)",165.0,2.22,22.0,1.39,30.0,1.80
6,"(pair-argmin,)",158.0,2.13,21.0,1.33,36.0,2.15
7,"(diff,)",150.0,2.02,45.0,2.84,21.0,1.26
8,"(argmin,)",80.0,1.08,19.0,1.20,15.0,0.90
9,"(topk-argmax,)",49.0,0.66,10.0,0.63,7.0,0.42


| Aggregation | Train | Test | Dev |
|-------------|-------|------|-----|
| [None]      | 5208  | 1133 | 1195 |
| Khác [None] | 2209  | 538  | 389  |


In [ ]:
aggregation = comparison_table['aggregation'].unique().tolist()
print(aggregation)

[('none',), ('pair-argmax',), ('div',), ('opposite',), ('argmax',), ('sum',), ('pair-argmin',), ('diff',), ('argmin',), ('topk-argmax',), ('greater_than',), ('max',), ('sum', 'div'), ('less_than',), ('min',), ('average',), ('kth-argmax',), ('diff', 'div'), ('range',), ('topk-argmin',), ('div', 'sum'), ('counta',), ('kth-argmin',), ('argmax', 'argmin'), ('max', 'min'), ('diff', 'opposite'), ('diff', 'greater_than'), ('sum', 'diff'), ('average', 'argmin'), ('sum', 'average', 'diff'), ('diff', 'topk-argmax'), ('sum', 'argmax'), ('diff', 'kth-argmax'), ('diff', 'min'), ('sum', 'pair-argmax'), ('sum', 'diff', 'div'), ('sum', 'opposite'), ('diff', 'topk-argmin'), ('counta', 'pair-argmax'), ('counta', 'less_than'), ('argmin', 'argmax'), ('average', 'div'), ('div', 'sum', 'pair-argmax'), ('range', 'diff'), ('average', 'topk-argmax'), ('div', 'diff'), ('average', 'argmax'), ('average', 'pair-argmax'), ('diff', 'argmax'), ('argmax', 'max'), ('argmin', 'min'), ('opposite', 'kth-argmax'), ('sum', 

Ví dụ cụ thể

In [ ]:
# @title
data = pd.concat([train_df, dev_df, test_df], ignore_index=True)
aggregation_groups = data.groupby('aggregation')['answer_formulas']

aggregation_analysis = {
    agg: pd.Series([str(item) for item in group]).unique()[:10]
    for agg, group in aggregation_groups
}

result_df = pd.DataFrame([
    {"Aggregation": agg, "Formulas": list(formulas[:10])}
    for agg, formulas in aggregation_analysis.items()
])
pd.set_option('display.max_colwidth', None)
result_df

,Aggregation,Formulas
0,"(argmax,)","[['=A10'], ['=A20'], ['=A11'], ['=A17'], ['=A14'], ['=D4'], ['=A16'], ['=E3'], ['=D3'], ['=B4']]"
1,"(argmax, argmin)","[['Edmonton', 'Calgary', 'Vancouver', 'Toronto', 'Windsor'], ['C7,C8'], ['B19,B5'], ['=A4', '=A8'], ['=A4', '=A12']]"
2,"(argmax, average)","[['=A9', '=B3']]"
3,"(argmax, max)","[['=A9', '=D9']]"
4,"(argmax, opposite)","[['=IF(F10=MIN(F5:F18),""Quebec"","""")']]"
5,"(argmin,)","[['=A13'], ['=A9', '=A11'], ['=B3'], ['=F4'], ['=A19'], ['=A23'], ['=A17'], ['=A20'], ['=D3'], ['=A10']]"
6,"(argmin, argmax)","[['=C14', '=C9']]"
7,"(argmin, min)","[['=A7', '=D7']]"
8,"(average,)","[['=(E11+E12+E13)/3'], ['=AVERAGE(B5:M5)'], ['=(G16+H16+I16+J16+K16+L16)/6'], ['=(G24+H24+I24+J24+K24+L24)/6'], ['=(F5+F6+F7+F8+F9+F10+F11+F12+F13+F14)/10'], ['=(B5+B6+B7+B8+B9+B10+B11+B12+B13+B14)/10'], ['=AVERAGE(B5:B15)'], ['=AVERAGE(D5:D15)'], ['=AVERAGE(C5:C15)'], ['=(F7+G7)/2']]"
9,"(average, argmax)",[['=F3']]


Lấy 'table_id', 'question', 'answer', 'reference_cells_map', 'table_df'



In [ ]:
# @title
def load_table_data(table_id):
    table_path = os.path.join(tables_dir, f"{table_id}.json")
    try:
        with open(table_path, "r", encoding="utf-8") as f:
            table_data = json.load(f)
        return table_data
    except FileNotFoundError:
        print(f"Table file for table_id {table_id} not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error decoding JSON for table_id {table_id}.")
        return None

def map_table(df):
    df['table_js'] = df['table_id'].apply(lambda x: load_table_data(x) if pd.notna(x) else None)
    return df

def convert_table(table_data):
    if table_data is None:
        return None
    texts = table_data.get("texts", [])
    return texts

# Hàm để chuyển đổi tất cả bảng trong DataFrame mà không cần `top_root`, `left_root`
def convert_tables(df):
    df['table_df'] = df['table_js'].apply(convert_table)
    return df
df = map_table(df)

df = convert_tables(df)

def extract_coordinates(ref_map):
    if ref_map:
        return [tuple(map(int, coords.strip("()").split(", "))) for coords in ref_map.values()]
    return []

df_filtered = df[['table_id', 'question', 'answer', 'reference_cells_map', 'table_df']].copy()
df_filtered['reference_cells_map'] = df_filtered['reference_cells_map'].apply(extract_coordinates)

df_filtered.head()


In [ ]:
df_filtered.to_csv("combined_dataframe.csv", index=False, encoding="utf-8")


## Tiền xử lý

In [ ]:
#Chuẩn hóa 2 lần answer

# Hàm chuẩn hóa bước đầu
def normalize_answer(answer):
    try:
        # Chuyển từ chuỗi JSON-like thành danh sách hoặc giá trị đơn
        answer = literal_eval(answer)
    except (ValueError, SyntaxError):
        answer = str(answer).strip()

    # Nếu là danh sách, nối các phần tử thành chuỗi với dấu phẩy
    if isinstance(answer, list):
        return ", ".join(str(a).strip() for a in answer)
    return str(answer).strip()

# Đọc tệp CSV
file_path = '/content/drive/MyDrive/Colab Notebooks/combined_dataframe.csv'
df = pd.read_csv(file_path)

# Chuẩn hóa bước đầu cột `answer`
df['answer'] = df['answer'].apply(normalize_answer)


In [ ]:
# Hàm chuẩn hóa bước tiếp theo
def normalize_answer2(answer):
    try:
        # Chuyển từ chuỗi thành số nếu có thể
        answer = float(answer) if '.' in answer else int(answer)
    except ValueError:
        return str(answer).strip()

    # Nếu là số
    if isinstance(answer, (int, float)):
        if isinstance(answer, float) and answer.is_integer():  # Nếu là float nhưng không có phần thập phân
            answer = int(answer)  # Chuyển thành int
        if answer >= 1000:  # Nếu lớn hơn 1000
            return f"{answer:,}"  # Định dạng dấu phẩy
        return str(answer)  # Chuyển về chuỗi
    return str(answer).strip()

# Áp dụng chuẩn hóa chi tiết hơn
df['answer'] = df['answer'].apply(normalize_answer2)


In [ ]:
df['answer'].iloc[0:20]

,answer
0,51
1,1
2,"8,189"
3,men
4,"cameroon, cote d'ivoire, senegal"
5,black population
6,29.2
7,9.8
8,caribbean and latin america
9,caribbean and latin america


In [ ]:
df[['table_id','question','answer','reference_cells_map']].to_csv("new.csv", index=False, encoding="utf-8")

## Thông tin sau khi xử lý

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/new.csv'
df = pd.read_csv(file_path)
tables_dir = "/content/drive/MyDrive/Colab Notebooks/Hitab/data/tables/csv"

# Hàm load bảng từ table_id
def load_table_data(table_id, tables_dir):
    table_file = os.path.join(tables_dir, f"{table_id}.csv")
    if not os.path.exists(table_file):
        print(f"Không tìm thấy file bảng: {table_file}")
        return None

    try:
        table_data = pd.read_csv(table_file, dtype=str).fillna("NaN")
        return table_data
    except Exception as e:
        print(f"Lỗi khi load bảng {table_id}: {e}")
        return None

# Lấy thông tin cơ bản và hiển thị chỉ các bảng cần thiết
for _, row in df.iloc[0:5].iterrows():
    table_id = row['table_id']
    question = row['question']
    answer = row['answer']
    reference_cells_map = row['reference_cells_map']

    print(f"Table ID: {table_id}")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f"Reference Cells Map: {reference_cells_map}")

    # Hiển thị bảng tương ứng
    table_data = load_table_data(table_id, tables_dir)
    if table_data is not None:
        print("Bảng tương ứng:")
        display(table_data)
    else:
        print("Không thể hiển thị bảng tương ứng.")

Table ID: 0_totto74-1
Question: in 2016 season, how many rushes did charles sims finish for 149 yards?
Answer: 51
Reference Cells Map: [(4, 4)]
Bảng tương ứng:


,season,team,games,games.1,rushing,rushing.1,rushing.2,rushing.3,rushing.4,receiving,receiving.1,receiving.2,receiving.3,receiving.4,fumbles,fumbles.1
0,season,team,games,games,rushing,rushing,rushing,rushing,rushing,receiving,receiving,receiving,receiving,receiving,fumbles,fumbles
1,NaN,NaN,gp,gs,att,yds,avg,lng,td,rec,yds,avg,lng,td,fum,lost
2,2014,tampa bay buccaneers,8,0,66,185,2.8,20,1,19,190,10.0,24,0,2,1
3,2015,tampa bay buccaneers,16,0,107,529,4.9,59,0,51,561,11.0,56,4,2,2
4,2016,tampa bay buccaneers,7,2,51,149,2.9,23,1,24,190,7.9,33,1,1,1
5,2017,tampa bay buccaneers,16,0,21,95,4.5,21,0,35,249,7.1,25,1,1,1
6,total,NaN,47,2,245,958,3.9,59,2,129,"1,190",9.2,56,6,6,5


Table ID: 0_totto109-3
Question: how many games did schwartz, the manager of rot-weiß erfurt between 11 april 2003 and 30 june 2003, won out of 10 matches?
Answer: 1
Reference Cells Map: [(2, 4)]
Bảng tương ứng:


,team,from,to,record,record.1,record.2,record.3,record.4,record.5
0,team,from,to,record,record,record,record,record,record
1,NaN,NaN,NaN,g,w,d,l,win %,ref
2,rot-weiß erfurt,11 april 2003,30 june 2003,10,1,3,6,10.00,none
3,1. fc kaiserslautern ii,1 january 2007,9 september 2012,193,86,48,59,44.56,none
4,1. fc kaiserslautern,4 may 2009,30 june 2009,4,1,0,3,25.00,none
5,rot-weiß erfurt,10 september 2012,30 september 2013,30,10,9,11,33.33,none
6,sv sandhausen,1 july 2013,25 june 2016,108,35,30,43,32.41,none
7,1. fc nurnberg,25 june 2016,7 march 2017,25,8,6,11,32.00,none
8,karlsruher sc,29 august 2017,present,46,23,15,8,50.00,none
9,total,NaN,NaN,416,164,111,141,39.42,-


Table ID: 0_totto115-4
Question: in total, how many rushing yards did craig finish during his eleven nfl seasons?
Answer: 8,189
Reference Cells Map: [(13, 3)]
Bảng tương ứng:


,none,rushing,rushing.1,rushing.2,rushing.3,rushing.4,rushing.5,receiving,receiving.1,receiving.2,receiving.3,receiving.4
0,none,rushing,rushing,rushing,rushing,rushing,rushing,receiving,receiving,receiving,receiving,receiving
1,year,team,att,yds,avg,lng,td,no,yds,avg,lng,td
2,1983,sf,176,725,4.1,71,8,48,427,8.9,23,4
3,1984,sf,155,649,4.2,28,4,71,675,9.5,64,3
4,1985,sf,214,"1,050",4.9,62,9,92,"1,016",11.0,73,6
5,1986,sf,204,830,4.1,25,7,81,624,7.7,48,0
6,1987,sf,215,815,3.8,25,3,66,492,7.5,35,1
7,1988,sf,310,"1,502",4.8,46,9,76,534,7.0,22,1
8,1989,sf,271,"1,054",3.9,27,6,49,473,9.7,44,1
9,1990,sf,141,439,3.1,26,1,25,201,8.0,31,0


Table ID: 1000
Question: which had a higher proportion of admittances under the family sponsorship program,women or men?
Answer: men
Reference Cells Map: [(14, 0)]
Bảng tương ứng:


,country of birth,economic immigrants,economic immigrants.1,economic immigrants.2,family-sponsored immigrants,refugees,other immigrants,total
0,country of birth,economic immigrants,economic immigrants,economic immigrants,family-sponsored immigrants,refugees,other immigrants,total
1,NaN,principal applicants,secondary applicants,total,NaN,NaN,NaN,NaN
2,NaN,percent,NaN,NaN,NaN,NaN,NaN,NaN
3,women,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,jamaica,31.2,9.0,40.2,55.9,2.0,2.0,100.0
5,haiti,26.3,12.5,38.8,35.8,18.6,6.8,100.0
6,trinidad and tobago,32.9,10.0,42.9,50.7,2.8,3.5,100.0
7,guyana,23.0,11.4,34.4,57.7,3.8,4.2,100.0
8,barbados,38.1,6.1,44.2,47.6,2.7,5.6,100.0
9,saint vincent and the grenadines,15.4,1.0,16.4,38.7,26.2,18.7,100.0


Table ID: 1001
Question: list african countries of origin that have more than half of the male immigrants admitted as a principal applicant under the economic program.
Answer: cameroon, cote d'ivoire, senegal
Reference Cells Map: [(33, 0), (36, 0), (37, 0)]
Bảng tương ứng:


,country of birth,economic immigrants,economic immigrants.1,economic immigrants.2,family-sponsored immigrants,refugees,other immigrants,total
0,country of birth,economic immigrants,economic immigrants,economic immigrants,family-sponsored immigrants,refugees,other immigrants,total
1,NaN,principal applicants,secondary applicants,total,NaN,NaN,NaN,NaN
2,NaN,percent,NaN,NaN,NaN,NaN,NaN,NaN
3,women,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ethiopia,3.0,3.6,6.6,34.9,58.0,0.5,100.0
5,somalia,1.0,1.1,2.1,21.3,75.1,1.5,100.0
6,eritrea,1.1,2.0,3.1,23.9,72.9,0.1,100.0
7,sudan,4.2,6.3,10.6,23.9,65.0,0.6,100.0
8,south sudan,0.0,0.4,0.4,14.5,85.1,0.0,100.0
9,burundi,3.9,9.5,13.4,16.8,64.6,5.2,100.0


# Encode và Finetune


## tokenize của tapex là 1024

Do tapex chỉ dùng được tối đa 1024 tokenize nên lọc ra những row nào lớn hơn 1024 thì bị loại



In [ ]:
import os
import pandas as pd
from transformers import TapexTokenizer

tables_dir = "/content/drive/MyDrive/Colab Notebooks/Hitab/data/tables/csv"
model_name = "microsoft/tapex-large"
tokenizer = TapexTokenizer.from_pretrained(model_name)

file_path = '/content/drive/MyDrive/Colab Notebooks/new.csv'
df = pd.read_csv(file_path)

# Hàm chuẩn hóa bảng
def normalize_table(table):
    """
    Chuẩn hóa bảng để đảm bảo tất cả các giá trị là chuỗi.
    """
    if not isinstance(table, pd.DataFrame):
        raise ValueError("Input table must be a pandas DataFrame.")
    return table.astype(str)

# Hàm tính tổng số token
def calculate_total_tokens(table, question, tokenizer):
    """
    Tính tổng số token cho một bảng và câu hỏi.
    """
    try:
        tokenized = tokenizer(
            table=table,
            query=question,
            padding="max_length",
            truncation=False,
            return_tensors="pt",
        )
        total_tokens = tokenized["input_ids"].shape[1]  # Số token cho mỗi truy vấn
        return total_tokens
    except Exception as e:
        print(f"Error during tokenization: {e}")
        return None

errors = []
results = []


for idx, row in df.iterrows():
    table_id = row.get("table_id", None)
    question = row.get("question", None)

    if pd.isna(table_id) or pd.isna(question):
        errors.append({"row_index": idx, "table_id": table_id, "error": "Missing table_id or question"})
        continue

    table_file_path = f"{tables_dir}/{table_id}.csv"

    try:
        table = pd.read_csv(table_file_path, dtype=str).fillna("")
        table = normalize_table(table)

        # Tính tổng số token
        total_tokens = calculate_total_tokens(table, question, tokenizer)

        if total_tokens is None:
            errors.append({"row_index": idx, "table_id": table_id, "error": "Tokenization failed"})
        else:
            results.append({"row_index": idx, "table_id": table_id, "total_tokens": total_tokens})

    except FileNotFoundError:
        errors.append({"row_index": idx, "table_id": table_id, "error": "Table file not found"})
    except Exception as e:
        errors.append({"row_index": idx, "table_id": table_id, "error": str(e)})

print("=== Results ===")
for result in results:
    print(f"Row Index: {result['row_index']}, Table ID: {result['table_id']}, Total Tokens: {result['total_tokens']}")

print("\n=== Errors ===")
for error in errors:
    print(f"Row Index: {error['row_index']}, Table ID: {error['table_id']}, Error: {error['error']}")


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Row Index: 5674, Table ID: 443_totto50982-2, Total Tokens: 1024
Row Index: 5675, Table ID: 443_totto51026-4, Total Tokens: 1024
Row Index: 5676, Table ID: 444_totto51076-0, Total Tokens: 1024
Row Index: 5677, Table ID: 444_totto51110-3, Total Tokens: 1024
Row Index: 5678, Table ID: 444_totto51147-4, Total Tokens: 1024
Row Index: 5679, Table ID: 446_totto51260-1, Total Tokens: 1024
Row Index: 5680, Table ID: 446_totto51354-3, Total Tokens: 1024
Row Index: 5681, Table ID: 447, Total Tokens: 1024
Row Index: 5682, Table ID: 447, Total Tokens: 1024
Row Index: 5683, Table ID: 447_totto51371-1, Total Tokens: 1024
Row Index: 5684, Table ID: 448_totto51433-0, Total Tokens: 1026
Row Index: 5685, Table ID: 449_totto51603-1, Total Tokens: 1024
Row Index: 5686, Table ID: 449_totto51627-2, Total Tokens: 1024
Row Index: 5687, Table ID: 449_totto51648-3, Total Tokens: 1024
Row Index: 5688, Table ID: 44_161_tab1, Total Tokens: 1024
Row Index: 568

Lưu những row nhỏ hơn hoặc bằng 1024<br>
và những row lớn hơn 1024


In [ ]:
over_1024 = [result for result in results if result['total_tokens'] > 1024]
equal_1024 = [result for result in results if result['total_tokens'] <= 1024]

df_over_1024 = pd.DataFrame(over_1024)
df_equal_1024 = pd.DataFrame(equal_1024)

df_over_1024.to_csv('over1024_tokens.csv', index=False)
df_equal_1024.to_csv('1024_tokens.csv', index=False)

print("Results saved:")
print("- Rows with tokens > 1024 saved to 'over1024_tokens.csv.csv'")
print("- Rows with tokens <= 1024 saved to '1024_tokens.csv'")


Results saved:
- Rows with tokens > 1024 saved to 'over1024_tokens.csv.csv'
- Rows with tokens <= 1024 saved to '1024_tokens.csv'


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Colab Notebooks/new.csv'
output_file_equal_1024 = '/content/drive/MyDrive/Colab Notebooks/equal1024.csv'
original_data = pd.read_csv(file_path)

# Liên kết `df_equal_1024` với file gốc bằng cột `table_id`
merged_data = original_data[original_data['table_id'].isin(df_equal_1024['table_id'])]
merged_data.to_csv(output_file_equal_1024, index=False)

print(f"Tạo thành công file mới: {output_file_equal_1024}")


Tạo thành công file mới: /content/drive/MyDrive/Colab Notebooks/equal1024.csv


In [ ]:
equal1024 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/equal1024.csv')
equal1024.shape

(8649, 4)

In [ ]:
import pandas as pd

equal1024 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/equal1024.csv')
equal1024_table_ids = equal1024['table_id'].unique()

data = pd.concat([train_df, dev_df, test_df], ignore_index=True)

filtered_data = data[data['table_id'].isin(equal1024_table_ids)]

aggregation_counts = filtered_data['aggregation'].value_counts()
aggregation_percentage = (aggregation_counts / aggregation_counts.sum()) * 100

result = pd.DataFrame({
    'Counts': aggregation_counts,
    'Percentage': aggregation_percentage
})
result


,Counts,Percentage
aggregation,,
"(none,)",6254,71.959498
"(pair-argmax,)",533,6.132781
"(div,)",354,4.073179
"(opposite,)",290,3.336785
"(argmax,)",246,2.830514
"(sum,)",189,2.174663
"(pair-argmin,)",180,2.071108
"(diff,)",161,1.852491
"(argmin,)",81,0.931999


Load và kiểm tra data



In [ ]:
import os
import pandas as pd

file_path = '/content/drive/MyDrive/Colab Notebooks/equal1024.csv'
tables_dir = '/content/drive/MyDrive/Colab Notebooks/Hitab/data/tables/csv'

def load_and_check_data(file_path, tables_dir):
    """
    Hàm đọc và kiểm tra dữ liệu từ file CSV.
    Kiểm tra tính hợp lệ của các bảng và dữ liệu đầu vào.
    """
    df = pd.read_csv(file_path)
    tables = []
    queries = []
    answers = []

    print(f"Đang kiểm tra file: {file_path}")
    print(f"Tổng số dòng trong dữ liệu: {len(df)}")

    for index, row in df.iterrows():
        table_id = row.get('table_id', None)
        query = row.get('question', None)
        answer = row.get('answer', None)

        # Kiểm tra tính hợp lệ của các giá trị trong dòng
        if not table_id or not query or not answer:
            print(f"Dòng {index} bị thiếu dữ liệu: {row}")
            continue


        table_file = os.path.join(tables_dir, f"{table_id}.csv")
        if not os.path.exists(table_file):
            print(f"Bảng không tồn tại: {table_file}, bỏ qua dòng {index}")
            continue

        try:

            table_df = pd.read_csv(table_file, dtype=str).fillna("")
        except Exception as e:
            print(f"Lỗi khi đọc bảng {table_file}: {e}")
            continue

        # Kiểm tra bảng hợp lệ
        if table_df.empty or table_df.shape[0] == 0 or table_df.shape[1] == 0:
            print(f"Bảng không hợp lệ: {table_file}, bỏ qua dòng {index}")
            continue

        if not isinstance(table_df, pd.DataFrame):
            print(f"Bảng không phải DataFrame: {table_file}, bỏ qua dòng {index}")
            continue

        tables.append(table_df)
        queries.append(query)
        answers.append(answer)

    print(f"Tổng số bảng hợp lệ: {len(tables)}")
    return {"table": tables, "query": queries, "answer": answers}

data = load_and_check_data(file_path, tables_dir)

def preview_data(data, num_samples=5):
    """
    In trước một số mẫu dữ liệu để kiểm tra trực quan.
    """
    print("\n=== Xem trước dữ liệu ===")
    for i in range(min(num_samples, len(data["table"]))):
        print(f"\nMẫu {i + 1}:")
        print("Bảng:")
        print(data["table"][i])
        print("Câu hỏi:")
        print(data["query"][i])
        print("Câu trả lời:")
        print(data["answer"][i])

preview_data(data, num_samples=5)


Đang kiểm tra file: /content/drive/MyDrive/Colab Notebooks/equal1024.csv
Tổng số dòng trong dữ liệu: 8649
Tổng số bảng hợp lệ: 8649

=== Xem trước dữ liệu ===

Mẫu 1:
Bảng:
   season                  team  games games.1  rushing rushing.1 rushing.2  \
0  season                  team  games   games  rushing   rushing   rushing   
1                                   gp      gs      att       yds       avg   
2    2014  tampa bay buccaneers      8       0       66       185       2.8   
3    2015  tampa bay buccaneers     16       0      107       529       4.9   
4    2016  tampa bay buccaneers      7       2       51       149       2.9   
5    2017  tampa bay buccaneers     16       0       21        95       4.5   
6   total                           47       2      245       958       3.9   

  rushing.3 rushing.4  receiving receiving.1 receiving.2 receiving.3  \
0   rushing   rushing  receiving   receiving   receiving   receiving   
1       lng        td        rec         yds      

## Tokenize và xử lý label


In [ ]:
import os
import pandas as pd
from transformers import TapexTokenizer
import torch
import pickle

tables_dir = "/content/drive/MyDrive/Colab Notebooks/Hitab/data/tables/csv"
model_name = "microsoft/tapex-large"
tokenizer = TapexTokenizer.from_pretrained(model_name)

file_path = "/content/drive/MyDrive/Colab Notebooks/equal1024.csv"

# Hàm chuẩn hóa dữ liệu
def sanitize_data(file_path, tables_dir):
    df = pd.read_csv(file_path)
    sanitized_tables = []
    sanitized_queries = []
    sanitized_answers = []

    for idx, row in df.iterrows():
        table_id = row.get('table_id', None)
        query = row.get('question', None)
        answer = row.get('answer', None)

        if not table_id or not query or not answer:
            print(f"Dòng {idx} bị thiếu dữ liệu: {row}")
            continue

        table_file = os.path.join(tables_dir, f"{table_id}.csv")
        if not os.path.exists(table_file):
            print(f"Bảng không tồn tại: {table_file}, bỏ qua dòng {idx}")
            continue

        try:
            table = pd.read_csv(table_file, dtype=str).fillna("")
        except Exception as e:
            print(f"Lỗi khi đọc bảng {table_file}: {e}")
            continue

        if not isinstance(table, pd.DataFrame) or table.empty:
            print(f"Bảng không hợp lệ: {table_file}, bỏ qua dòng {idx}")
            continue

        if not isinstance(answer, str):
            answer = str(answer) if answer is not None else "unknown"

        sanitized_tables.append(table)
        sanitized_queries.append(query)
        sanitized_answers.append(answer)

    return {"table": sanitized_tables, "query": sanitized_queries, "answer": sanitized_answers}

data = sanitize_data(file_path, tables_dir)
print("=== Dữ liệu đã chuẩn hóa ===")
print(f"Tổng số bảng hợp lệ: {len(data['table'])}")

# Mã hóa bảng và câu hỏi
def encode_table_and_query(data, tokenizer):
    input_encodings = []
    for idx, (table, query) in enumerate(zip(data["table"], data["query"])):
        try:
            encoding = tokenizer(
                table=table,
                query=query,
                truncation=True,
                padding="max_length",
                return_tensors="pt"
            )
            input_encodings.append(encoding)
            print(f"Mẫu {idx + 1}: Thành công! input_ids shape: {encoding['input_ids'].shape}")
        except Exception as e:
            print(f"Lỗi khi mã hóa bảng và câu hỏi tại mẫu {idx}: {e}")
            continue
    return input_encodings

input_encodings = encode_table_and_query(data, tokenizer)
print("\n=== Hoàn thành mã hóa bảng và câu hỏi ===")
print(f"Tổng số mẫu được mã hóa: {len(input_encodings)}")

# Mã hóa nhãn (labels)
def encode_labels_with_padding(data, tokenizer, padding_value=0):
    """
    Mã hóa các nhãn (labels) và thực hiện padding để các tensor cùng kích thước.
    """
    labels = []
    for idx, answer in enumerate(data["answer"]):
        try:
            # Tokenize nhãn
            label = tokenizer(
                answer=answer,
                return_tensors="pt"
            ).input_ids.squeeze(0)
            labels.append(label)
            print(f"Mẫu {idx + 1}: Thành công! label shape: {label.shape}")
        except Exception as e:
            print(f"Lỗi khi mã hóa nhãn tại mẫu {idx}: {e}")
            continue

    # Thực hiện padding để các nhãn có cùng kích thước
    try:
        padded_labels = pad_sequence(labels, batch_first=True, padding_value=padding_value)
        print(f"Padding hoàn tất! Kích thước padded labels: {padded_labels.shape}")
        return padded_labels
    except Exception as e:
        print(f"Lỗi khi thực hiện padding: {e}")
        return None

# Gọi hàm mã hóa nhãn với padding
padded_label_encodings = encode_labels_with_padding(data, tokenizer, padding_value=tokenizer.pad_token_id)

print("\n=== Hoàn thành mã hóa nhãn ===")
print(f"Tổng số nhãn được mã hóa: {len(padded_label_encodings)}")

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Mẫu 3654: Thành công! label shape: torch.Size([4])
Mẫu 3655: Thành công! label shape: torch.Size([5])
Mẫu 3656: Thành công! label shape: torch.Size([5])
Mẫu 3657: Thành công! label shape: torch.Size([3])
Mẫu 3658: Thành công! label shape: torch.Size([5])
Mẫu 3659: Thành công! label shape: torch.Size([5])
Mẫu 3660: Thành công! label shape: torch.Size([5])
Mẫu 3661: Thành công! label shape: torch.Size([5])
Mẫu 3662: Thành công! label shape: torch.Size([5])
Mẫu 3663: Thành công! label shape: torch.Size([5])
Mẫu 3664: Thành công! label shape: torch.Size([5])
Mẫu 3665: Thành công! label shape: torch.Size([3])
Mẫu 3666: Thành công! label shape: torch.Size([5])
Mẫu 3667: Thành công! label shape: torch.Size([6])
Mẫu 3668: Thành công! label shape: torch.Size([5])
Mẫu 3669: Thành công! label shape: torch.Size([5])
Mẫu 3670: Thành công! label shape: torch.Size([5])
Mẫu 3671: Thành công! label shape: torch.Size([5])
Mẫu 3672: Thành công! lab

In [ ]:
output_file = "/content/drive/MyDrive/Colab Notebooks/encoded_data.pkl"
with open(output_file, "wb") as f:
    pickle.dump({"input_encodings": input_encodings, "labels": padded_label_encodings}, f)
print(f"Dữ liệu mã hóa đã được lưu tại: {output_file}")


Dữ liệu mã hóa đã được lưu tại: /content/drive/MyDrive/Colab Notebooks/encoded_data.pkl


In [ ]:
import pickle
with open("/content/drive/MyDrive/Colab Notebooks/encoded_data.pkl", "rb") as f:
    data = pickle.load(f)

input_encodings = data["input_encodings"]
padded_label_encodings = data["labels"]

print(f"Tổng số input_encodings: {len(input_encodings)}")
print(f"Tổng số labels: {len(padded_label_encodings)}")

Tổng số input_encodings: 8649
Tổng số labels: 8649


Tạo train, dev, test


In [ ]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu thành train (70%), còn lại là tạm (30%)
train_input, temp_input, train_labels, temp_labels = train_test_split(
    input_encodings, padded_label_encodings, test_size=0.3, random_state=42
)

# Chia tập tạm thành dev (50%) và test (50%)
dev_input, test_input, dev_labels, test_labels = train_test_split(
    temp_input, temp_labels, test_size=0.5, random_state=42
)

print(f"Số lượng mẫu train: {len(train_input)}")
print(f"Số lượng mẫu dev: {len(dev_input)}")
print(f"Số lượng mẫu test: {len(test_input)}")

Số lượng mẫu train: 6054
Số lượng mẫu dev: 1297
Số lượng mẫu test: 1298


## Tự Fine tune TAPEX


In [ ]:
from transformers import BartForConditionalGeneration, TapexTokenizer
import torch
from torch.optim import AdamW
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score

# Tải mô hình TAPEX và tokenizer
model_name = "microsoft/tapex-base"
tokenizer = TapexTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()

epochs = 3
batch_size = 8

# Hàm tính loss trên tập dev
def evaluate_dev_set(dev_input, dev_labels):
    model.eval()
    dev_loss = 0
    with torch.no_grad():
        for i in range(0, len(dev_input), batch_size):
            input_batch = dev_input[i:i + batch_size]
            label_batch = dev_labels[i:i + batch_size]

            input_ids = torch.cat([encoding["input_ids"] for encoding in input_batch], dim=0).to(device)
            attention_mask = torch.cat([encoding["attention_mask"] for encoding in input_batch], dim=0).to(device)
            labels_tensor = pad_sequence(label_batch, batch_first=True, padding_value=tokenizer.pad_token_id).to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_tensor)
            dev_loss += outputs.loss.item()

    model.train()
    return dev_loss / len(dev_input)

# Vòng lặp huấn luyện
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    epoch_loss = 0

    for i in range(0, len(train_input), batch_size):
        input_batch = train_input[i:i + batch_size]
        label_batch = train_labels[i:i + batch_size]

        input_ids = torch.cat([encoding["input_ids"] for encoding in input_batch], dim=0).to(device)
        attention_mask = torch.cat([encoding["attention_mask"] for encoding in input_batch], dim=0).to(device)
        labels_tensor = pad_sequence(label_batch, batch_first=True, padding_value=tokenizer.pad_token_id).to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_tensor)
        loss = outputs.loss
        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} completed. Training Loss: {epoch_loss / len(train_input)}")

    # Tính loss trên tập dev
    dev_loss = evaluate_dev_set(dev_input, dev_labels)
    print(f"Dev Loss after Epoch {epoch + 1}: {dev_loss}")

model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

print("Đã lưu mô hình và tokenizer vào thư mục 'fine_tuned_model'.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/988 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Epoch 1/3
Epoch 1 completed. Training Loss: 0.028466138084485837
Dev Loss after Epoch 1: 0.011405067341710939
Epoch 2/3
Epoch 2 completed. Training Loss: 0.010201930303956056
Dev Loss after Epoch 2: 0.011507363552651272
Epoch 3/3
Epoch 3 completed. Training Loss: 0.008166506118610315
Dev Loss after Epoch 3: 0.010741401142774634


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'num_beams': 4, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Đã lưu mô hình và tokenizer vào thư mục 'fine_tuned_model'.


In [ ]:
import torch
from transformers import BartForConditionalGeneration, TapexTokenizer


model_name = "fine_tuned_model"
save_directory = "/content/drive/MyDrive/Colab Notebooks/fine_tuned_model"

tokenizer = TapexTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Đã lưu mô hình và tokenizer vào thư mục: {save_directory}")


Đã lưu mô hình và tokenizer vào thư mục: /content/drive/MyDrive/Colab Notebooks/fine_tuned_model


## Kiểm tra trên tập test


In [ ]:
import torch
from transformers import BartForConditionalGeneration, TapexTokenizer
import pickle
from sklearn.model_selection import train_test_split

model_name = "fine_tuned_model"
tokenizer = TapexTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

with open("/content/drive/MyDrive/Colab Notebooks/encoded_data.pkl", "rb") as f:
    data = pickle.load(f)

input_encodings = data["input_encodings"]
padded_label_encodings = data["labels"]

# Chia dữ liệu thành train, dev và test
train_input, temp_input, train_labels, temp_labels = train_test_split(input_encodings, padded_label_encodings, test_size=0.3, random_state=42)
dev_input, test_input, dev_labels, test_labels = train_test_split(temp_input, temp_labels, test_size=0.5, random_state=42)

print(f"Số lượng mẫu train: {len(train_input)}")
print(f"Số lượng mẫu dev: {len(dev_input)}")
print(f"Số lượng mẫu test: {len(test_input)}")

# Dự đoán trên tập kiểm thử
predictions = []
true_answers = []
correct_count = 0

for idx, encoding in enumerate(test_input):
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50, no_repeat_ngram_size=3, num_beams=4, early_stopping=True)

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    true_answer = tokenizer.decode(test_labels[idx], skip_special_tokens=True)
    predictions.append(prediction)
    true_answers.append(true_answer)

    is_correct = prediction.strip().lower() == true_answer.strip().lower()
    if is_correct:
        correct_count += 1
    print(f"Mẫu {idx + 1}: Dự đoán: {prediction}, Đáp án gốc: {true_answer}, {'Đúng' if is_correct else 'Sai'}")


accuracy = correct_count / len(test_input) * 100
print(f"Độ chính xác: {accuracy:.2f}%")



Số lượng mẫu train: 6054
Số lượng mẫu dev: 1297
Số lượng mẫu test: 1298
Mẫu 1: Dự đoán: apolonia fier, Đáp án gốc: apolonia fier, Đúng
Mẫu 2: Dự đoán: 85.4, Đáp án gốc: 22.9, Sai
Mẫu 3: Dự đoán: 11, Đáp án gốc: 11, Đúng
Mẫu 4: Dự đoán: 46, Đáp án gốc: 46, Đúng
Mẫu 5: Dự đoán: construction trade helpers and labourers, transport truck drivers, Đáp án gốc: light duty cleaners, cashiers, Sai
Mẫu 6: Dự đoán: 27, Đáp án gốc: 27, Đúng
Mẫu 7: Dự đoán: 70.8, Đáp án gốc: 70.8, Đúng
Mẫu 8: Dự đoán: 167, Đáp án gốc: 167, Đúng
Mẫu 9: Dự đoán: 223, Đáp án gốc: 223, Đúng
Mẫu 10: Dự đoán: 1,781, Đáp án gốc: 1,781, Đúng
Mẫu 11: Dự đoán: 7.7, Đáp án gốc: 4.7, Sai
Mẫu 12: Dự đoán: 41.7, Đáp án gốc: 41.7, Đúng
Mẫu 13: Dự đoán: 31,727, Đáp án gốc: 31,427, Sai
Mẫu 14: Dự đoán: 9.8, Đáp án gốc: 9.8, Đúng
Mẫu 15: Dự đoán: 80, Đáp án gốc: 80, Đúng
Mẫu 16: Dự đoán: 25 to 34, Đáp án gốc: 1999.0, 2005.0, 2016.0, Sai
Mẫu 17: Dự đoán: 13.5, Đáp án gốc: 13.744444, Sai
Mẫu 18: Dự đoán: 0.078, Đáp án gốc: 0.076, Sai
M

TAPEX Fine-tunned WTQ

In [ ]:
import torch
from transformers import BartForConditionalGeneration, TapexTokenizer
import pickle
from sklearn.model_selection import train_test_split

model_name = "microsoft/tapex-large-finetuned-wtq"
tokenizer = TapexTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

with open("/content/drive/MyDrive/Colab Notebooks/encoded_data.pkl", "rb") as f:
    data = pickle.load(f)

input_encodings = data["input_encodings"]
padded_label_encodings = data["labels"]

# Chia dữ liệu thành train, dev và test
train_input, temp_input, train_labels, temp_labels = train_test_split(input_encodings, padded_label_encodings, test_size=0.3, random_state=42)
dev_input, test_input, dev_labels, test_labels = train_test_split(temp_input, temp_labels, test_size=0.5, random_state=42)

print(f"Số lượng mẫu train: {len(train_input)}")
print(f"Số lượng mẫu dev: {len(dev_input)}")
print(f"Số lượng mẫu test: {len(test_input)}")

# Dự đoán trên tập kiểm thử
predictions = []
true_answers = []
correct_count = 0

for idx, encoding in enumerate(test_input):
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50, no_repeat_ngram_size=3, num_beams=4, early_stopping=True)

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    true_answer = tokenizer.decode(test_labels[idx], skip_special_tokens=True)
    predictions.append(prediction)
    true_answers.append(true_answer)

    is_correct = prediction.strip().lower() == true_answer.strip().lower()
    if is_correct:
        correct_count += 1
    print(f"Mẫu {idx + 1}: Dự đoán: {prediction}, Đáp án gốc: {true_answer}, {'Đúng' if is_correct else 'Sai'}")


accuracy = correct_count / len(test_input) * 100
print(f"Độ chính xác: {accuracy:.2f}%")


Số lượng mẫu train: 6054
Số lượng mẫu dev: 1297
Số lượng mẫu test: 1298
Mẫu 1: Dự đoán:  apolonia fier, Đáp án gốc: apolonia fier, Đúng
Mẫu 2: Dự đoán:  85.4, Đáp án gốc: 22.9, Sai
Mẫu 3: Dự đoán:  11, Đáp án gốc: 11, Đúng
Mẫu 4: Dự đoán:  36, Đáp án gốc: 46, Sai
Mẫu 5: Dự đoán:  light duty cleaners, cashiers, Đáp án gốc: light duty cleaners, cashiers, Đúng
Mẫu 6: Dự đoán:  32, Đáp án gốc: 27, Sai
Mẫu 7: Dự đoán:  70.8, Đáp án gốc: 70.8, Đúng
Mẫu 8: Dự đoán:  16, Đáp án gốc: 167, Sai
Mẫu 9: Dự đoán:  94, Đáp án gốc: 223, Sai
Mẫu 10: Dự đoán:  1,781, Đáp án gốc: 1,781, Đúng
Mẫu 11: Dự đoán:  7.7, Đáp án gốc: 4.7, Sai
Mẫu 12: Dự đoán:  %, Đáp án gốc: 41.7, Sai
Mẫu 13: Dự đoán:  1707, Đáp án gốc: 31,427, Sai
Mẫu 14: Dự đoán:  30, Đáp án gốc: 9.8, Sai
Mẫu 15: Dự đoán:  40, Đáp án gốc: 80, Sai
Mẫu 16: Dự đoán:  2005, 2006, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, Đáp án gốc: 1999.0, 2005.0, 2016.0, Sai
Mẫu 17: Dự đoán:  1.625, Đáp án gốc: 13.744444, Sai
Mẫu 18: Dự đoán:  0.076,

TAPAS Fine-tunned WTQ

In [ ]:
import os
import pandas as pd
from transformers import TapasTokenizer, TapasForQuestionAnswering
import torch
from sklearn.model_selection import train_test_split
import pickle

# Đường dẫn file
file_path = '/content/drive/MyDrive/Colab Notebooks/equal1024.csv'
tables_dir = '/content/drive/MyDrive/Colab Notebooks/Hitab/data/tables/csv'
checkpoint_file = '/content/drive/MyDrive/Colab Notebooks/tapas_checkpoint.pkl'

# Tải tokenizer và mô hình Tapas
tapas_model_name = "google/tapas-large-finetuned-wtq"
tokenizer_tapas = TapasTokenizer.from_pretrained(tapas_model_name)
model_tapas = TapasForQuestionAnswering.from_pretrained(tapas_model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_tapas.to(device)

# Hàm chuẩn hóa dữ liệu từ file CSV
def sanitize_data(file_path, tables_dir):
    df = pd.read_csv(file_path)
    sanitized_tables = []
    sanitized_queries = []
    sanitized_answers = []

    for idx, row in df.iterrows():
        table_id = row.get('table_id', None)
        query = row.get('question', None)
        answer = row.get('answer', None)

        if not table_id or not query or not answer:
            print(f"Dòng {idx} bị thiếu dữ liệu: {row}")
            continue

        table_file = os.path.join(tables_dir, f"{table_id}.csv")
        if not os.path.exists(table_file):
            print(f"Bảng không tồn tại: {table_file}, bỏ qua dòng {idx}")
            continue

        try:
            table = pd.read_csv(table_file, dtype=str).fillna("")
        except Exception as e:
            print(f"Lỗi khi đọc bảng {table_file}: {e}")
            continue

        if not isinstance(table, pd.DataFrame) or table.empty:
            print(f"Bảng không hợp lệ: {table_file}, bỏ qua dòng {idx}")
            continue

        if not isinstance(answer, str):
            answer = str(answer) if answer is not None else "unknown"

        sanitized_tables.append(table)
        sanitized_queries.append(query)
        sanitized_answers.append(answer)

    return {"table": sanitized_tables, "query": sanitized_queries, "answer": sanitized_answers}

# Chuẩn hóa dữ liệu
data = sanitize_data(file_path, tables_dir)

# Chia dữ liệu theo tỷ lệ 70:15:15
train_tables, temp_tables, train_queries, temp_queries, train_answers, temp_answers = train_test_split(
    data["table"], data["query"], data["answer"], test_size=0.3, random_state=42)

dev_tables, test_tables, dev_queries, test_queries, dev_answers, test_answers = train_test_split(
    temp_tables, temp_queries, temp_answers, test_size=0.5, random_state=42)

# Load checkpoint nếu tồn tại
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "rb") as f:
        checkpoint = pickle.load(f)
        start_idx = checkpoint.get("start_idx", 0)
        correct_count = checkpoint.get("correct_count", 0)
else:
    start_idx = 0
    correct_count = 0

# Inference trên tập test
for idx in range(start_idx, len(test_tables)):
    table = test_tables[idx]
    query = test_queries[idx]
    true_answer = test_answers[idx]

    # Mã hóa dữ liệu trong thời gian thực
    inputs = tokenizer_tapas(
        table=table,
        queries=[query],
        truncation=True,
        padding="max_length",
        max_length=1024,
        return_tensors="pt"
    )

    # Chuyển dữ liệu lên thiết bị
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Dự đoán
    with torch.no_grad():
        outputs = model_tapas(**inputs)

    # Lấy câu trả lời từ logits
    predicted_table_cell_coords, _ = tokenizer_tapas.convert_logits_to_predictions(
        inputs,
        outputs.logits.cpu().detach(),
        outputs.logits_aggregation.cpu().detach()
    )

    # Trích xuất câu trả lời từ bảng
    predicted_answer = []
    for coords in predicted_table_cell_coords[0]:  # Chỉ một câu hỏi
        if coords:
            predicted_answer.append(table.iloc[coords[0], coords[1]])
    predicted_answer = " ".join(map(str, predicted_answer))

    # So sánh với đáp án
    is_correct = predicted_answer.strip().lower() == true_answer.strip().lower()
    correct_count += int(is_correct)
    print(f"Mẫu {idx + 1}: Dự đoán: {predicted_answer}, Đáp án gốc: {true_answer}, {'Đúng' if is_correct else 'Sai'}")

    # Lưu checkpoint mỗi 200 mẫu hoặc khi hoàn thành
    if (idx + 1) % 200 == 0 or idx == len(test_tables) - 1:
        checkpoint = {"start_idx": idx + 1, "correct_count": correct_count}
        with open(checkpoint_file, "wb") as f:
            pickle.dump(checkpoint, f)
        print(f"Checkpoint được lưu tại mẫu {idx + 1}")

# Tính độ chính xác
accuracy = correct_count / len(test_queries) * 100
print(f"Độ chính xác trên tập test: {accuracy:.2f}%")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Mẫu 801: Dự đoán: , Đáp án gốc: 2.46, Sai
Mẫu 802: Dự đoán: total guilty ipv cases, Đáp án gốc: accused males, Sai
Mẫu 803: Dự đoán: 25, Đáp án gốc: 25, Đúng
Mẫu 804: Dự đoán: 2013, Đáp án gốc: 1,957, Sai
Mẫu 805: Dự đoán: montreal - laval, Đáp án gốc: english-language workers, Sai
Mẫu 806: Dự đoán: portsmouth, Đáp án gốc: bournemouth, Sai
Mẫu 807: Dự đoán: 69.6, Đáp án gốc: 77, Sai
Mẫu 808: Dự đoán: 13, Đáp án gốc: 13, Đúng
Mẫu 809: Dự đoán: 5265, Đáp án gốc: 27, Sai
Mẫu 810: Dự đoán: china league one, Đáp án gốc: china league one, Đúng
Mẫu 811: Dự đoán: 13, Đáp án gốc: 13, Đúng
Mẫu 812: Dự đoán: 25,690, Đáp án gốc: 82,950, Sai
Mẫu 813: Dự đoán: - 28, Đáp án gốc: 28, Sai
Mẫu 814: Dự đoán: 1,449,220, Đáp án gốc: 14.5, Sai
Mẫu 815: Dự đoán: marseille, Đáp án gốc: marseille, Đúng
Mẫu 816: Dự đoán: canada, Đáp án gốc: quebec, Sai
Mẫu 817: Dự đoán: 74, Đáp án gốc: 204.8, Sai
Mẫu 818: Dự đoán: 6.2, Đáp án gốc: 9.4, Sai
Mẫu 819: Dự đoán: 50.7, Đáp án gốc: 53.2, Sai
Mẫu 820: Dự đoán: 84.3 14.

# Tổng quan

| Mô hình                     | Kết quả % |
|-----------------------------|---------|
| Tapex (fine-tuned WTQ)      | 29.04   |
| Tapas (fine-tuned WTQ)      | 24.50   |
| Tapex (tự fine-tuned)       | 45.07   |
